In [1]:
#original code https://github.com/enakai00/autoencoder_example

import tensorflow as tf
import os,shutil
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.python.estimator.model_fn import ModeKeys as Modes
from tensorflow.contrib.learn import Experiment
from tensorflow.contrib.learn.python.learn import learn_runner
from tensorflow.contrib.learn.python.learn.utils import (saved_model_export_utils)

tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR='/tmp/trained_model'

#
# definining queue
#
def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _,serialized_example = reader.read(filename_queue)
    
    features = tf.parse_single_example(
        serialized_example,
        features={
            'image_raw':tf.FixedLenFeature([],tf.string),
            'label':tf.FixedLenFeature([],tf.int64),
        })
    
    image = tf.decode_raw(features['image_raw'],tf.uint8)
    image.set_shape([784]) #image shape is (784,)
    image = tf.cast(image,tf.float32)*(1.0/255)
    label = tf.cast(features['label'],tf.int32)
    
    return image,label

def input_fn(filename,batch_size=100):
    filename_queue = tf.train.string_input_producer([filename])
    
    image,label = read_and_decode(filename_queue)
    images,labels = tf.train.batch(
        [image,label],batch_size=batch_size,
        capacity=1000+3*batch_size)
    #images : (100,784), labels : (100,1)
    
    return {'inputs':images},labels

def get_input_fn(filename,batch_size=100):
    return lambda: input_fn(filename,batch_size)

def serving_input_fn():
    inputs = {'inputs':tf.placeholder(tf.float32,[None,784])}
    return tf.estimator.export.ServingInputReceiver(inputs,inputs)

#
# define model
#
def autoencoder_model_fn(features,labels,mode):
    input_layer = features['inputs']
    dense1 = tf.layers.dense(inputs=input_layer,units=256,activation=tf.nn.relu)
    dense2 = tf.layers.dense(inputs=dense1,units=128,activation=tf.nn.relu)
    dense3 = tf.layers.dense(inputs=dense2,units=16,activation=tf.nn.relu)
    dense4 = tf.layers.dense(inputs=dense3,units=128,activation=tf.nn.relu)
    dense5 = tf.layers.dense(inputs=dense4,units=256,activation=tf.nn.relu)
    output_layer = tf.layers.dense(inputs=dense5,units=784,activation=tf.nn.sigmoid)
    
    #training and evaluation mode
    if mode in (Modes.TRAIN,Modes.EVAL):
        global_step = tf.contrib.framework.get_or_create_global_step()
        label_indices = tf.cast(labels,tf.int32)
        loss = tf.reduce_sum(tf.square(output_layer - input_layer))
        tf.summary.scalar('OptimizeLoss',loss)

        if mode == Modes.TRAIN:
            optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
            train_op = optimizer.minimize(loss,global_step=global_step)
            return tf.estimator.EstimatorSpec(mode,loss = loss, train_op = train_op)
        if mode == Modes.EVAL:
            eval_metric_ops = None
            return tf.estimator.EstimatorSpec(
                mode,loss=loss,eval_metric_ops = eval_metric_ops)
        
    # prediction mode
    if mode == Modes.PREDICT:
        predictions={
            'outputs':output_layer
        }
        export_outputs={
            'outputs':tf.estimator.export.PredictOutput(predictions)
        }
        return tf.estimator.EstimatorSpec(
            mode,predictions=predictions,export_outputs=export_outputs) #이부분 코드 상세 조사할것
    

def build_estimator(model_dir):
    return tf.estimator.Estimator(
        model_fn = autoencoder_model_fn,
        model_dir = model_dir,
        config=tf.contrib.learn.RunConfig(save_checkpoints_secs=180))

def generate_experiment_fn(data_dir,
                          train_batch_size = 100,
                          eval_batch_size = 100,
                          train_steps = 1000,
                          eval_steps = 1,
                          **experiment_args):
    def _experiment_fn(output_dir):
        return Experiment(
            build_estimator(output_dir),
            train_input_fn=get_input_fn('./data/train.tfrecords',batch_size=train_batch_size),
            eval_input_fn=get_input_fn('./data/test.tfrecords',batch_size=eval_batch_size),
            export_strategies = [saved_model_export_utils.make_export_strategy(
                serving_input_fn,
                default_output_alternative_key=None,
                exports_to_keep=1)
            ],
            train_steps = train_steps,
            eval_steps = eval_steps,
            **experiment_args
        )
    return _experiment_fn

shutil.rmtree(OUTDIR, ignore_errors=True) # start fresh each time
learn_runner.run(
    generate_experiment_fn(
        data_dir='./data/',
        train_steps=2000),
    OUTDIR)
    

/Users/freddie/anaconda3/envs/tf-conda-3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
from IPython.display import HTML
HTML(input_form + javascript)

NameError: name 'sess' is not defined